# Homework Assignment 3: Additional Data Prep
As in the previous data prep assignment, in this homework assignment, you will continue your exploration of the [SWAN-SF Dataset](https://doi.org/10.7910/DVN/EBCFKM), described in the paper found [here](https://doi.org/10.1038/s41597-020-0548-x).


This assignment will have you utilize the data summary table you constructed for Homework Assignment 2, as well as the partially cleaned feature dataset that was left after the completion of that assignment. 

## Step 1: Downloading the Data

This assignment will continue to only use [Partition 1](https://dataverse.harvard.edu/api/access/datafile/:persistentId?persistentId=doi:10.7910/DVN/EBCFKM/BMXYCB). Recall that in Homework 1, we started to construct the analytics base table for our [SWAN-SF Dataset](https://doi.org/10.7910/DVN/EBCFKM). In that assignment, we read the data from the two subdirectories, __FL__ and __NF__, of the __partition1__ direcotry. These two subdirectories represented the two classes of our target feature in the solar flare prediction problem we will be attempting to solve this semester. We then processed these samples of multivariate time series to construct descriptive features for each sample, and then placed them into our analytics base table.

Then, in Homework 2, you utilized a set of extracted descriptive features much like what you were asked to construct in Homework 1. However, this dataset contained many more extracted features than you were asked to compute for Homework 1 (>800), so we needed to explore the data to find data quality issues and identify ways to address these issues. Below are the links to the full extracted feature for all of partition 1, and a toy representative dataset, that were used as input to Homework 2.

- [Full Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/partition1ExtractedFeatures.csv)
- [Toy Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/toy_partition1ExtractedFeatures.csv)

Below you will find copies of the data for this assignment. If you didn't complete the Homework 2 assignment, or are worried your solution doesn't match the solution provided, then utilize the links below to download the data you will use for this assignment. __Note:__ Since the full dataset, and multiple copies of partially processed intermediary results, tend to take up a bit of space, you can use the toy dataset to implement and test your code. Onece you have your code working, you will then want to run with the full partition dataset to get the plots to show all the information you need. To do that, you should go to the `Kernel` dropdown and select `Restart & Run All`.  

- [Full Cleaned Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/cleaned_partition1ExtractedFeatures.csv)
- [Toy Cleaned Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/toy_cleaned_partition1ExtractedFeatures.csv)
- [Data Quality Table for Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/data_quality_table.csv)

Now that you have the partially cleaned features and data quality table csv files, you will load that data into a Pandas DataFrame using the [pandas.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) method. 

In [ ]:
%matplotlib inline
import os
import math
from typing import List
import pandas as pd
from pandas import DataFrame 
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

### Read the dataset

In [ ]:
data_dir = '/data/MVTS'
summary_table_name = 'data_quality_table.csv'
data_file = 'cleaned_partition1ExtractedFeatures.csv'

In [ ]:
abt = pd.read_csv(os.path.join(data_dir, data_file))
summary_table = pd.read_csv(os.path.join(data_dir, summary_table_name))

### Q1 (10 points)
Let's start trying to identify features in our dataset that may be good candidates for decimal scaling. We might want to do this with features that have a large range between the Minimum and Maximum feature value. We should also be concerned with features thta have a large number of outliers, as we may wish to treat them differently than features with a large range but just a few outliers. This is because it may be the case that the feature with a large range and large number of outliers has a bimodal/multi-modal distribution where one mode dominates the population (not uncommon with highly imbalanced data).  

So, to begin our investigation, construct a function that takes in the Data Quality/Summary table, and returns the list of features that have a range between the Minimum and Maximum of more than 10,000, and also has more than 10% of the feature values as outliers.

After you complete the function, call it and maybe look over some of the features listed. We will be investigating a small number of them further in the next questions. 

In [ ]:
def get_features_with_large_range(summary_table:DataFrame) -> List[str]:
    return [row['Feature Name'] for i, row in summary_table.iterrows() if (row['Max']-row['Min']) > 10000 
    and ((row['Outlier Count Low'] + row['Outlier Count High']) / (row['Non-null Count'] + row['Null Count']))> 0.1]

In [ ]:
summary_table = pd.read_csv(os.path.join(data_dir, summary_table_name))

### Q2 (10 points)
Now that you have a list of the features that have a large range and a large number of outliers, let's investigate a few and see how they relate to one another, and how their distributions look.  If we see that they follow a normal distribution, then perhaps we would want to utilize Z-Score normalization instead of decimal or range normalization. Also, if we find that the outliers are not part of a bimodal/multi-modal distribution, maybe we will wish to perform some outlier processing strategy like clipping before we consider what scaling/normalization method to apply.

So, for this question, use the matplotlib [subplots](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.subplots.html) function to get the Figure and Axes object used to plot a set of figures. You need to request a 3 row by 1 column subplot grid to plot the three features `USFLUX_var`, `TOTUSJH_var`, and `TOTFZ_var`. You should also make the figsize different than default to make sure you can see the subfigures properly.  So, add `figsize=(8,21)` as one of the arguments to the function call as well. 

Once that is done, you can use the DataFrame [hist](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.hist.html) method to plot a 20 bin histogram on your Axes object returned from your subplots call described above. This means you will pass the Axes object into this method call.

In [ ]:
features_to_look_at = ['USFLUX_var','TOTUSJH_var','TOTFZ_var']

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(8, 10))
hist = abt[features_to_look_at].hist(bins=20,figsize=(10, 21))

### Q3 (15 points)
If you got yor histogram plots to work for Q2 above, you would see that each of these features are highly right skewed, almost exponential like distribution.  But what does the distribution look like if we clamp the high outliers to some value, like the threshold for being classified as an outlier on the high end or $Q3+1.5IQR$.

So, for this question, let's do that. Construct a function that performs clamping of the high outliers in place (meaning pass in a DataFrame with only the three features you want to process and assign the new values back to that input DataFrame). The high outliers should be clamped to the threshold value that we used to classify them as an outlier $val > Q3+1.5IQR$. Once you have done this, then replot the features from Q2 after applying this funciton on them.

In [ ]:
def clamp_to_third_quartile(data:DataFrame) -> None:
    for key in list(data):
        data.loc[:, key].clip(upper=data[key].quantile(0.75)+((data[key].quantile(0.75)-data[key].quantile(0.25))*1.50), inplace=True)

In [ ]:
df = abt[features_to_look_at]
clamp_to_third_quartile(df)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(8, 10))
hist = df.hist(bins=20,figsize=(10, 21))

### Q4 (10 points)
Now that we have handled the outliers in these three features, the distributions look a bit different. You can now definitely see that the values follow an exponental distribution, at least up to the threshold we clamped the outliers to. We can also see that `USFLUX_var` and `TOTFZ_var` still look to be good candidates for decimal scaling, where `TOTUSJH_var` could probably now use range scaling.  

These outliers being clamped to an upper threshold may or may not be helpful for our final classification task. If all of the outlier values happen to be one class, or could be split to one class based on a the combination of the features, then we would like to keep them like this. If not, then maybe we consider other features for investigation.  

Let's investigate further by plotting a Scatter Plot Matrix (pair plot) of these features. To accomplish this, use the seaborn [pairplot](https://seaborn.pydata.org/generated/seaborn.pairplot.html) function. You should also add the labels to your DataFrame before passing it to the function. To add the labels (recall they are called `lab` in the dataset), use the DataFrame [join](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html) method to join a DataFrame containing the labels and your DataFrame containing the partially processed features from Q4. When calling the pairplot method, you will then use the `hue='lab'` argument to tell it to plot each label as different color, so we can see how the different classes are distributed.  Additionally, you should use the `diag_kind='hist'` argument to specify that the diagonal subplots are histogram plots. 

In [ ]:
df = df.join(abt['lab'])
sns.pairplot(df, hue='lab', diag_kind='hist')

### Q5 (15 points)
If you were to plot the three features using the entire partition, you would see that the combination of `USFLUX_var` and `TOTFZ_var` seem to push most of the `M` and `X` class flares to the upper right hand corner of the pair plot whereas the use of `TOTUSJH_var` doesn't make this as obvious when paired with either of the other two features.  So, let's assume that the combination of `USFLUX_var` and `TOTFZ_var` is a good first set of features and get them ready for use.

For this question, let's start to construct our dataset of cleaned and scaled/normalized feature values. Since we decided that `USFLUX_var` and `TOTFZ_var` are candidates for use later, let's processe them. We saw from our plots above, that even with the outliers clipped, they both may benefit from decimal scaling. So, lets construct a function to process these two. 

First, construct a function that takes in the entire analytics base table of raw feature values and a feature name (I've started the function below). The function then returns the clipped and decimal scaled feature as a DataFrame. The high outliers should be clamped to the threshold value that we used to classify them as an outlier $val > Q3+1.5IQR$ (Call your function from Q3 to do this to save having to rewrite code). Decimal scaling should then be applied such that $v_{i}'= \frac{v_{i}}{10^{j}}$ where $j$ is the smallest integer such that $max(|v_{i}'|)<1$ or $j=log_{10}(max(|v|))[take\ the\ ceiling]$.

Next, call this function twice, each time passing in the analytics base table, and one of the feature names `USFLUX_var` or `TOTFZ_var`. Then join the two returned feature sets to construct the start of our cleaned feature table (keep this around for later you will be adding to it). Additionally, make sure to join the labels to your resultant DataFrame. 

In [ ]:
def clip_upper_and_decimal_scale(data:DataFrame, feature_name:str) -> DataFrame:
    df = pd.DataFrame(data[feature_name])
    clamp_to_third_quartile(df)
    for i in range(len(df)):
        df.iloc[i] = df.iloc[i]/pow(10, math.ceil(math.log(df.max(), 10)))

    return df 

In [ ]:
df1 = clip_upper_and_decimal_scale(abt, 'USFLUX_var')
df2 = clip_upper_and_decimal_scale(abt, 'TOTUSJH_var')
df = pd.concat([pd.DataFrame(abt['lab']), df1, df2], axis=1)

### Q6 (10 points)
Let's start trying to identify features in our dataset that may be good candidates for Z-Score normalization. We might want to do this with features that are the opposite of the ones we found for Q1, namely ones that don't have a large number of outliers.

So, to begin our investigation, construct a function like was done for Q1, one that takes in the Data Quality/Summary table, and returns the list of features that meet our criteria. For this question, we wish to to list those that have a range between the Minimum and Maximum of less than 10,000, and also have less than 5% of the feature values as outliers.

Once you complete the function, call it and maybe investigate the list of features.  We will work with a small number in the next questions.

In [ ]:
def get_features_with_smaller_range(summary_table:DataFrame) -> List[str]:
    return [row['Feature Name'] for i, row in summary_table.iterrows() if (row['Max']-row['Min']) < 10000 
    and ((row['Outlier Count Low'] + row['Outlier Count High']) / (row['Non-null Count'] + row['Null Count']))< 0.05]

In [ ]:
get_features_with_smaller_range(summary_table)

### Q7 (10 points)
Now that you have a list of the features that have a smaller range and a smaller number of outliers, lets investigate a few and see how they relate to one another, and how their distributions look.  If we see that they follow a normal distribution, then perhaps this is a good indication that we would want to utilize Z-Score normalization.  We will still want to handle outliers, but hopefully this won't be as much of an issue as they were with other data we processed in earlier questions.

So, like you did for Q2, use the matplotlib [subplots](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.subplots.html) function to get the Figure and Axes object used to plot a set of figures. You need to request a 3 row by 1 column subplot grid to plot the three features listed below. You should also make the figsize different than default to make sure you can see the subfigures.  So, add `figsize=(8,21)` as one of the arguments to the function call as well. 

Once that is done, you can use the DataFrame [hist](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.hist.html) method to plot a 20 bin histogram on your Axes object returned from your subplots call described above. This means you will pass the Axes object into this method call. Plot the features `R_VALUE_median`, `MEANGBT_median`, and `TOTUSJH_skewness` to view how they are distributed.

In [ ]:
small_outlier_features = ['R_VALUE_median', 'MEANGBT_median', 'TOTUSJH_skewness']

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(8, 10))
hist = abt[small_outlier_features].hist(bins=20,figsize=(10, 21))

### Q8 (10 points)
If you got yor histogram plots to work above using the full partition dataset, you would see `MEANGBT_median` and `TOTUSJH_skewness` have distributions that look to be similar to a normal distribution.  So, these are probably good candidates for Z-Score normalization. On the other hand, `R_VALUE_median` is more of a bimodal distribution with a large number zero values, and a second more normal looking distribution.  So, this one is probably not a good candidate for Z-Score, and we will simply use range normalization on this feature (which will be applied to all features prior to being sent to various classifiers, but we'll cover that in another assignment). 

So, for this question, you will perform Z-Score normalization on the two features `MEANGBT_median` and `TOTUSJH_skewness`, and then join them and `R_VALUE_median` to the cleaned feature table you made for Q6.  

First, construct a function that takes in the entire analytics base table of raw feature values and a feature name (I've started the function below). The function then returns the Z-Score normalized feature requested by the input name as a DataFrame. This function should simply apply Z-Score normalizaiton to a copy of the feature from the raw feature values table using the $a_{i}' = \frac{a_{i}-\hat{a}}{sd(a)}$ where $\hat{a}$ is the mean of the feature and $sd(a)$ is the standard deviation of the feature. 

Next, call this function twice by passing in the analytics base table, and the each of the feature names `MEANGBT_median` and `TOTUSJH_skewness`. Then join the two returned feature sets to the cleaned feature table from Q6. Additionally, you will also need to join the `R_VALUE_median` to the cleaned feature table from Q6. 

In [ ]:
def do_z_score_normalization(data:DataFrame, feature_name:str) -> DataFrame:
    df = pd.DataFrame(data[feature_name])
    for i in range(len(df)):
        df.iloc[i] = (df.iloc[i]-df.mean())/df.std()

    return df

In [ ]:
v3 = do_z_score_normalization(abt, 'MEANGBT_median')
v4 = do_z_score_normalization(abt, 'TOTUSJH_skewness')
df = pd.concat([df, v3, v4, abt['R_VALUE_median']], axis=1)

### Q9 (10 points)
After Q9, we now have a cleaned feature table with 5 different features, lets's see how well the combinations of each pair of features does in separating the different classes of target feature. To do this, let's again plot a Scatter Plot Matrix (pair plot) of these features like was done in Q5. 

Like before, use the seaborn [pairplot](https://seaborn.pydata.org/generated/seaborn.pairplot.html) function. When calling the pairplot method, you will use `hue='lab'` argument to tell it to plot each label as different color, so we can see how the different classes are distributed.  Additionally, you should use the `diag_kind='hist'` to specify that the diagonal subplots are histogram plots. 

To answer the questions below, you will need to plot the full partition so you should use the drop down menu `Kernel` -> `Restart & Run All` after you change your input data to point to the full partition. This will rerun all of the code above and apply all of the functions and join all of the features you have been asked to join to this point. Then use the output plots to answer these questions in another cell below:
#### Q9.a - Is there any feature (or combination there of) you think would work well to partition the dataset into X,M and C,B,NF binary groups?
#### Q9.b - What threshold(s) would work well to accomplish the task of Q10.a?

In [ ]:
sns.pairplot(df, hue='lab', diag_kind='hist')

In [ ]:
Q9.a the small outlier features do a good job of giving the data a shape that can be worked well with
q9.b the decimal scaling threshold does a good job of grouping the data into clusters so you can seee where the data sits
